# NYC Rodent Merged Dataset
### 2018-2023

#### _Overview_ 
This notebook merges aggregated datasets on rodent sightings, sanitation complaints, subways (todo) and restaurant inspections (todo).

In [1]:
from datetime import datetime, timedelta
import geopandas as gpd
import json
import pandas as pd
import mapclassify
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from io import StringIO
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)
np.set_printoptions(threshold=np.inf)

In [2]:
# Parameters
parent_dir = os.path.abspath('..')  # get the absolute path of the parent directory

## Read Aggregated (cleaned) Data Sources

In [3]:
# Read Census Block Groups
cb_file_path = os.path.join(parent_dir, 'Data', 'nyc_bgrp.geojson')  # construct the file path
cb_gdf = gpd.read_file(cb_file_path)  # load the GeoJSON file into a GeoDataFrame
print(cb_gdf.dtypes)
print(len(cb_gdf))
cb_gdf.head(2)


spatial_id                    object
name                          object
%OfOwnerOccupiedHousing      float64
%OfRenterOccupiedHousing     float64
MedianHouseholdIncome        float64
TotalPopulation              float64
geometry                    geometry
dtype: object
6493


,spatial_id,name,%OfOwnerOccupiedHousing,%OfRenterOccupiedHousing,MedianHouseholdIncome,TotalPopulation,geometry
0,360050001000,"BG0001000, Bronx County, NY",0.0,0.0,NaN,0.0,"MULTIPOLYGON (((-73.89679 40.79616, -73.89681 ..."
1,360050001001,"BG0001001, Bronx County, NY",0.0,0.0,NaN,6472.0,"POLYGON ((-73.89076 40.79782, -73.89076 40.797..."


In [4]:
# Read Rodent Sightings (311)
sightings_file_path = os.path.join(parent_dir, 'Data', 'sightings_per_year.csv')  # construct the file path
sightings_df = pd.read_csv(sightings_file_path)
sightings_df['spatial_id'] = sightings_df['spatial_id'].astype(str)
print(len(sightings_df))
print(sightings_df.dtypes)
sightings_df.head(2)


38958
spatial_id                  object
year                         int64
l_Commercial_sum           float64
l_Other_sum                float64
l_Outdoor_sum              float64
l_Residential_sum          float64
l_Residential-Mixed_sum    float64
l_Vacant_Space_sum         float64
d_Friday_sum               float64
d_Monday_sum               float64
d_Saturday_sum             float64
d_Sunday_sum               float64
d_Thursday_sum             float64
d_Tuesday_sum              float64
d_Wednesday_sum            float64
t_Evening_sum              float64
t_Midday_sum               float64
t_Morning_sum              float64
num_sightings              float64
dtype: object


,spatial_id,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,d_Saturday_sum,d_Sunday_sum,d_Thursday_sum,d_Tuesday_sum,d_Wednesday_sum,t_Evening_sum,t_Midday_sum,t_Morning_sum,num_sightings
0,360050001000,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050001001,2020,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0


In [5]:
# Read DSNY Complaints
dsny_complaints_file_path = os.path.join(parent_dir, 'Data', 'dsny_complaints_per_year.csv')  # construct the file path
dsny_complaints_df = pd.read_csv(dsny_complaints_file_path)
dsny_complaints_df['spatial_id'] = dsny_complaints_df['spatial_id'].astype(str)
print(len(dsny_complaints_df))
print(dsny_complaints_df.dtypes)
dsny_complaints_df.head(2)


38958
spatial_id                           object
year                                  int64
s_Dead_Animal:Residential_sum       float64
s_Dead_Animal:Street_sum            float64
s_Dog_waste:Street_sum              float64
s_Illegal_Dumping:Street_sum        float64
s_Trash:Residential_sum             float64
s_Trash:Street_sum                  float64
s_Trash_MissedService:Street_sum    float64
s_Trash_Overflowing:Street_sum      float64
s_Trash_Time:Street_sum             float64
s_Trash_Unsecure:Residential_sum    float64
s_Trash_Unsecure:Street_sum         float64
num_dsny_complaints                 float64
dtype: object


,spatial_id,year,s_Dead_Animal:Residential_sum,s_Dead_Animal:Street_sum,s_Dog_waste:Street_sum,s_Illegal_Dumping:Street_sum,s_Trash:Residential_sum,s_Trash:Street_sum,s_Trash_MissedService:Street_sum,s_Trash_Overflowing:Street_sum,s_Trash_Time:Street_sum,s_Trash_Unsecure:Residential_sum,s_Trash_Unsecure:Street_sum,num_dsny_complaints
0,360050001000,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050001001,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Merge Data

In [14]:
cbg_merged_gdf = pd.merge(cb_gdf, sightings_df, on=['spatial_id'], how='left')

print(len(cbg_merged_gdf))
cbg_merged_gdf.head(2)

38958


,spatial_id,name,%OfOwnerOccupiedHousing,%OfRenterOccupiedHousing,MedianHouseholdIncome,TotalPopulation,geometry,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,d_Saturday_sum,d_Sunday_sum,d_Thursday_sum,d_Tuesday_sum,d_Wednesday_sum,t_Evening_sum,t_Midday_sum,t_Morning_sum,num_sightings
0,360050001000,"BG0001000, Bronx County, NY",0.0,0.0,NaN,0.0,"MULTIPOLYGON (((-73.89679 40.79616, -73.89681 ...",2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050001000,"BG0001000, Bronx County, NY",0.0,0.0,NaN,0.0,"MULTIPOLYGON (((-73.89679 40.79616, -73.89681 ...",2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
cbg_merged_gdf = pd.merge(cbg_merged_gdf, dsny_complaints_df, on=['spatial_id','year'], how='left')

print(len(cbg_merged_gdf))
cbg_merged_gdf.head(2)

38958


,spatial_id,name,%OfOwnerOccupiedHousing,%OfRenterOccupiedHousing,MedianHouseholdIncome,TotalPopulation,geometry,year,l_Commercial_sum,l_Other_sum,l_Outdoor_sum,l_Residential_sum,l_Residential-Mixed_sum,l_Vacant_Space_sum,d_Friday_sum,d_Monday_sum,d_Saturday_sum,d_Sunday_sum,d_Thursday_sum,d_Tuesday_sum,d_Wednesday_sum,t_Evening_sum,t_Midday_sum,t_Morning_sum,num_sightings,s_Dead_Animal:Residential_sum_x,s_Dead_Animal:Street_sum_x,s_Dog_waste:Street_sum_x,s_Illegal_Dumping:Street_sum_x,s_Trash:Residential_sum_x,s_Trash:Street_sum_x,s_Trash_MissedService:Street_sum_x,s_Trash_Overflowing:Street_sum_x,s_Trash_Time:Street_sum_x,s_Trash_Unsecure:Residential_sum_x,s_Trash_Unsecure:Street_sum_x,num_dsny_complaints_x,s_Dead_Animal:Residential_sum_y,s_Dead_Animal:Street_sum_y,s_Dog_waste:Street_sum_y,s_Illegal_Dumping:Street_sum_y,s_Trash:Residential_sum_y,s_Trash:Street_sum_y,s_Trash_MissedService:Street_sum_y,s_Trash_Overflowing:Street_sum_y,s_Trash_Time:Street_sum_y,s_Trash_Unsecure:Residential_sum_y,s_Trash_Unsecure:Street_sum_y,num_dsny_complaints_y,s_Dead_Animal:Residential_sum,s_Dead_Animal:Street_sum,s_Dog_waste:Street_sum,s_Illegal_Dumping:Street_sum,s_Trash:Residential_sum,s_Trash:Street_sum,s_Trash_MissedService:Street_sum,s_Trash_Overflowing:Street_sum,s_Trash_Time:Street_sum,s_Trash_Unsecure:Residential_sum,s_Trash_Unsecure:Street_sum,num_dsny_complaints
0,360050001000,"BG0001000, Bronx County, NY",0.0,0.0,NaN,0.0,"MULTIPOLYGON (((-73.89679 40.79616, -73.89681 ...",2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,360050001000,"BG0001000, Bronx County, NY",0.0,0.0,NaN,0.0,"MULTIPOLYGON (((-73.89679 40.79616, -73.89681 ...",2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Normalize columns by Population

In [ ]:
# TODO 

## Save Merged Dataset

In [20]:
export_columns = ['spatial_id'] + cbg_merged_gdf.columns[7:].to_list()
print(export_columns)
parent_dir = os.path.abspath('..')  # get the absolute path of the parent directory
file_path = os.path.join(parent_dir, 'Data', 'rodents_per_year_merged.csv')  # construct the file path
cbg_merged_gdf[export_columns].to_csv(file_path, index=False)

['spatial_id', 'year', 'l_Commercial_sum', 'l_Other_sum', 'l_Outdoor_sum', 'l_Residential_sum', 'l_Residential-Mixed_sum', 'l_Vacant_Space_sum', 'd_Friday_sum', 'd_Monday_sum', 'd_Saturday_sum', 'd_Sunday_sum', 'd_Thursday_sum', 'd_Tuesday_sum', 'd_Wednesday_sum', 't_Evening_sum', 't_Midday_sum', 't_Morning_sum', 'num_sightings', 's_Dead_Animal:Residential_sum_x', 's_Dead_Animal:Street_sum_x', 's_Dog_waste:Street_sum_x', 's_Illegal_Dumping:Street_sum_x', 's_Trash:Residential_sum_x', 's_Trash:Street_sum_x', 's_Trash_MissedService:Street_sum_x', 's_Trash_Overflowing:Street_sum_x', 's_Trash_Time:Street_sum_x', 's_Trash_Unsecure:Residential_sum_x', 's_Trash_Unsecure:Street_sum_x', 'num_dsny_complaints_x', 's_Dead_Animal:Residential_sum_y', 's_Dead_Animal:Street_sum_y', 's_Dog_waste:Street_sum_y', 's_Illegal_Dumping:Street_sum_y', 's_Trash:Residential_sum_y', 's_Trash:Street_sum_y', 's_Trash_MissedService:Street_sum_y', 's_Trash_Overflowing:Street_sum_y', 's_Trash_Time:Street_sum_y', 's_Tra